Import Dependacies

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Load and preprocess the data
df = pd.read_csv('../data/lawas/lawas-rainfall-daily.csv', encoding='latin')
df["DateTime"] = pd.to_datetime(df["DateTime"],format="%Y-%m-%d")
df.set_index("DateTime",inplace=True)

Feature Engineering (Lagged Day - 30 Days)

In [13]:
for lag in range(1, 31):
    df[f'Rainfall_lag_{lag}'] = df['Rainfall'].shift(lag)
    df[f'Temperature_lag_{lag}'] = df['Temperature'].shift(lag)
    df[f'Humidity_lag_{lag}'] = df['Humidity'].shift(lag)

In [14]:
# Scaling the features and target separately
feature_scaler = RobustScaler()
target_scaler = RobustScaler()

# Separate features and target (Rainfall)
features = df.drop(columns=['Rainfall']).reset_index(drop=True)
target = df['Rainfall']

# Scale features and target
features_scaled = feature_scaler.fit_transform(features)
target_scaled = target_scaler.fit_transform(target.values.reshape(-1, 1))

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    features_scaled, target_scaled, 
    test_size=0.30, random_state=100, shuffle=False
)

In [19]:
nn_model = Sequential()
nn_model.add(Dense(12, input_dim=x_train.shape[1], activation='relu'))
nn_model.add(Dense(1))
nn_model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1)
history = nn_model.fit(x_train, y_train, epochs=100, batch_size=1, verbose=1, callbacks=[early_stop], shuffle=False)

Epoch 1/100


c:\Users\UlamKetum\rainfall-forecast\ann\.virt\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 591us/step - loss: 2.3092
Epoch 2/100
4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 609us/step - loss: 2.1341
Epoch 3/100
4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 592us/step - loss: 2.1340
Epoch 4/100
4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 613us/step - loss: 2.1341
Epoch 5/100
4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 583us/step - loss: 2.1341
Epoch 6/100
4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 583us/step - loss: 2.1341
Epoch 7/100
4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 604us/step - loss: 2.1341
Epoch 8/100
4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 588us/step - loss: 2.1341
Epoch 9/100
4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 580us/step - loss: 2.1341
Epoch 10/100
4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 585us/step - loss: 2.1341
Epoch 11/100
4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 613us/step - loss: 2.1341
Epoch 12/100
4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 586us/step - loss: 2.1341
Epoch 13/100
4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 579us/step - loss: 2.1341
Epoch 14/100
4733/4733 ━━━━━━━━━━━━━━━━━━━━ 3s 592us/step - loss: 2.1341

In [22]:
y_pred_test_nn = nn_model.predict(x_test)
y_train_pred_nn = nn_model.predict(x_train)
print("The R2 score on the Train set is:\t{:0.3f}".format(r2_score(y_train, y_train_pred_nn)))
print("The R2 score on the Test set is:\t{:0.3f}".format(r2_score(y_test, y_pred_test_nn)))

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 427us/step
The R2 score on the Train set is:	-0.000
The R2 score on the Test set is:	-0.001
